# Image Captioning: Exploratory Data Analysis

This notebook performs exploratory data analysis on the Flickr8k dataset:
- Dataset statistics
- Caption analysis
- Image properties
- Vocabulary analysis

In [ ]:
# Setup and imports
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from collections import Counter
from wordcloud import WordCloud
from tqdm import tqdm

# Add project root to path
project_root = os.path.dirname(os.getcwd())
sys.path.append(project_root)

# Import project modules
from src.utils.manager import ConfigManager
from src.utils.config import load_config
from src.utils.constants import SEED
from src.preprocessing.vocabulary import preprocess_caption

# Set style
plt.style.use('default')

# Set random seed
np.random.seed(SEED)

In [ ]:
# Load configuration using ConfigManager
from src.utils.manager import ConfigManager

# Initialize configuration manager
config_manager = ConfigManager()

# Get configurations
data_config = config_manager.get_data_params()

# Get paths from config manager
captions_file = data_config['dataset']['captions_file']
images_dir = data_config['dataset']['images_dir']

print(f"Captions file: {captions_file}")
print(f"Images directory: {images_dir}")

# Check if files exist
print(f"\nCaptions file exists: {os.path.exists(captions_file)}")
print(f"Images directory exists: {os.path.exists(images_dir)}")

## Load and Explore Dataset

In [ ]:
# Note: captions_file path is already set from the previous cell
print(f"Loading from: {captions_file}")

In [ ]:
# Load captions
captions_df = pd.read_csv(captions_file)
print(f"Dataset shape: {captions_df.shape}")
print(f"\nColumns: {captions_df.columns.tolist()}")
print(f"\nFirst few rows:")
captions_df.head()

In [ ]:
# Basic statistics
num_images = len(captions_df['image'].unique())
num_captions = len(captions_df)
avg_captions_per_image = num_captions / num_images

print("Dataset Statistics:")
print(f"Total unique images: {num_images}")
print(f"Total captions: {num_captions}")
print(f"Average captions per image: {avg_captions_per_image:.2f}")

# Check caption distribution per image
caption_counts = captions_df['image'].value_counts()
print(f"\nCaptions per image distribution:")
print(caption_counts.value_counts().sort_index())

## Caption Analysis

In [ ]:
# Process captions
print("Processing captions...")
captions_df['processed_caption'] = captions_df['caption'].apply(preprocess_caption)
captions_df['caption_length'] = captions_df['processed_caption'].apply(lambda x: len(x.split()))

# Caption length statistics
print("\nCaption Length Statistics:")
print(captions_df['caption_length'].describe())

In [ ]:
# Caption length distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(captions_df['caption_length'], bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('Caption Length (words)')
plt.ylabel('Frequency')
plt.title('Distribution of Caption Lengths')
plt.grid(False)

plt.subplot(1, 2, 2)
captions_df['caption_length'].plot(kind='box')
plt.ylabel('Caption Length (words)')
plt.title('Caption Length Box Plot')
plt.grid(False)

plt.tight_layout()
plt.show()

In [ ]:
# Word frequency analysis
all_words = []
for caption in captions_df['processed_caption']:
    all_words.extend(caption.split())

word_freq = Counter(all_words)
print(f"Total unique words: {len(word_freq)}")
print(f"Total words: {len(all_words)}")

# Most common words
print("\nTop 20 most common words:")
for word, count in word_freq.most_common(20):
    print(f"{word}: {count} ({count/len(all_words)*100:.2f}%)")

In [ ]:
# Word cloud visualization
plt.figure(figsize=(15, 8))

# All words
plt.subplot(1, 2, 1)
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(all_words))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Word Cloud - All Words')
plt.axis('off')

# Without stopwords
plt.subplot(1, 2, 2)
# Filter out common words
stopwords = {'a', 'an', 'the', 'is', 'in', 'on', 'and', 'of', 'with', 'at', 'to', 'are'}
filtered_words = [w for w in all_words if w not in stopwords]
wordcloud_filtered = WordCloud(width=800, height=400, background_color='white').generate(' '.join(filtered_words))
plt.imshow(wordcloud_filtered, interpolation='bilinear')
plt.title('Word Cloud - Content Words')
plt.axis('off')

plt.tight_layout()
plt.show()

## Image Analysis

In [ ]:
# Sample images for analysis
sample_images = captions_df['image'].unique()[:100]  # Analyze first 100 images

# Collect image properties
image_properties = []

for img_name in tqdm(sample_images, desc="Analyzing images"):
    img_path = os.path.join(images_dir, img_name)
    if os.path.exists(img_path):
        img = Image.open(img_path)
        image_properties.append({
            'filename': img_name,
            'width': img.width,
            'height': img.height,
            'aspect_ratio': img.width / img.height,
            'mode': img.mode,
            'format': img.format
        })

image_df = pd.DataFrame(image_properties)

In [ ]:
# Image size distribution
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.scatter(image_df['width'], image_df['height'], alpha=0.5)
plt.xlabel('Width (pixels)')
plt.ylabel('Height (pixels)')
plt.title('Image Dimensions')
plt.grid(False)

plt.subplot(1, 3, 2)
plt.hist(image_df['aspect_ratio'], bins=20, edgecolor='black', alpha=0.7)
plt.xlabel('Aspect Ratio (width/height)')
plt.ylabel('Frequency')
plt.title('Aspect Ratio Distribution')
plt.grid(False)

plt.subplot(1, 3, 3)
image_df['resolution'] = image_df['width'] * image_df['height']
plt.hist(image_df['resolution'] / 1e6, bins=20, edgecolor='black', alpha=0.7)
plt.xlabel('Resolution (megapixels)')
plt.ylabel('Frequency')
plt.title('Resolution Distribution')
plt.grid(False)

plt.tight_layout()
plt.show()

print("Image Statistics:")
print(f"Average width: {image_df['width'].mean():.1f} pixels")
print(f"Average height: {image_df['height'].mean():.1f} pixels")
print(f"Average aspect ratio: {image_df['aspect_ratio'].mean():.2f}")
print(f"Image formats: {image_df['format'].value_counts().to_dict()}")

## Caption-Image Relationship

In [ ]:
# Analyze caption variation for same image
sample_image = captions_df['image'].unique()[0]
sample_captions = captions_df[captions_df['image'] == sample_image]

print(f"Sample image: {sample_image}")
print(f"\nCaptions for this image:")
for i, (_, row) in enumerate(sample_captions.iterrows(), 1):
    print(f"{i}. {row['caption']}")
    
# Display the image
img_path = os.path.join(images_dir, sample_image)
img = Image.open(img_path)
plt.figure(figsize=(8, 6))
plt.imshow(img)
plt.title(f"Image: {sample_image}")
plt.axis('off')
plt.show()

In [ ]:
# Caption diversity analysis
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Calculate caption similarity for images with multiple captions
similarity_scores = []

for img in tqdm(captions_df['image'].unique()[:100], desc="Calculating similarities"):
    img_captions = captions_df[captions_df['image'] == img]['processed_caption'].tolist()
    if len(img_captions) > 1:
        # Calculate TF-IDF vectors
        vectorizer = TfidfVectorizer()
        tfidf_matrix = vectorizer.fit_transform(img_captions)
        
        # Calculate pairwise similarities
        similarities = cosine_similarity(tfidf_matrix)
        
        # Get upper triangle (excluding diagonal)
        for i in range(len(img_captions)):
            for j in range(i + 1, len(img_captions)):
                similarity_scores.append(similarities[i, j])

# Plot similarity distribution
plt.figure(figsize=(10, 5))
plt.hist(similarity_scores, bins=30, edgecolor='black', alpha=0.7)
plt.xlabel('Cosine Similarity')
plt.ylabel('Frequency')
plt.title('Distribution of Caption Similarities for Same Image')
plt.grid(False)
plt.show()

print(f"Average caption similarity: {np.mean(similarity_scores):.3f}")
print(f"Std caption similarity: {np.std(similarity_scores):.3f}")

## Vocabulary Analysis with Different Thresholds

In [ ]:
# Analyze vocabulary size with different frequency thresholds
thresholds = [1, 2, 3, 4, 5, 10, 20]
vocab_sizes = []
coverage_rates = []

for threshold in thresholds:
    # Count words above threshold
    vocab_size = sum(1 for count in word_freq.values() if count >= threshold) + 4  # +4 for special tokens
    vocab_sizes.append(vocab_size)
    
    # Calculate coverage
    covered_words = sum(count for word, count in word_freq.items() if count >= threshold)
    coverage = covered_words / len(all_words) * 100
    coverage_rates.append(coverage)

# Plot results
fig, ax1 = plt.subplots(figsize=(10, 6))

color = 'tab:blue'
ax1.set_xlabel('Frequency Threshold')
ax1.set_ylabel('Vocabulary Size', color=color)
ax1.plot(thresholds, vocab_sizes, 'o-', color=color, linewidth=2, markersize=8)
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(False)

ax2 = ax1.twinx()
color = 'tab:red'
ax2.set_ylabel('Word Coverage (%)', color=color)
ax2.plot(thresholds, coverage_rates, 's-', color=color, linewidth=2, markersize=8)
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Vocabulary Size and Coverage vs Frequency Threshold')
fig.tight_layout()
plt.show()

# Print table
print("Threshold | Vocab Size | Coverage")
print()
for t, v, c in zip(thresholds, vocab_sizes, coverage_rates):
    print(f"{t:9d} | {v:10d} | {c:7.2f}%")

## Summary Statistics

In [ ]:
# Create summary report
print("DATASET SUMMARY")

print(f"\nImages:")
print(f"  Total images: {num_images}")
print(f"  Average resolution: {image_df['resolution'].mean() / 1e6:.1f} MP")
print(f"  Most common aspect ratio: {image_df['aspect_ratio'].mode()[0]:.2f}")

print(f"\nCaptions:")
print(f"  Total captions: {num_captions}")
print(f"  Captions per image: {avg_captions_per_image:.1f}")
print(f"  Average caption length: {captions_df['caption_length'].mean():.1f} words")
print(f"  Caption length range: {captions_df['caption_length'].min()}-{captions_df['caption_length'].max()} words")

print(f"\nVocabulary:")
print(f"  Total unique words: {len(word_freq)}")
print(f"  Words with freq >= 5: {sum(1 for c in word_freq.values() if c >= 5)}")
print(f"  Most common word: '{word_freq.most_common(1)[0][0]}' ({word_freq.most_common(1)[0][1]} times)")

print(f"\nCaption Diversity:")
print(f"  Average similarity between captions: {np.mean(similarity_scores):.3f}")
print(f"  This indicates {'high' if np.mean(similarity_scores) > 0.7 else 'moderate' if np.mean(similarity_scores) > 0.4 else 'low'} similarity")